In [2]:
import os
import services as _services



In [3]:
files_name = list(map(lambda x: os.path.basename(x).replace(".csv",""),os.listdir("data")))
files_name

['.DS_Store', 'hired_employees', 'departments', 'jobs']

In [4]:
file_paths = [f"data/{filename}.csv" for filename in _services.get_all_files("data")]
print(file_paths)


['data/hired_employees.csv', 'data/departments.csv', 'data/jobs.csv']


In [5]:
from fastapi import File, UploadFile
import pandas as pd
import sqlalchemy as sa
from pathlib import Path
from sqlalchemy import create_engine
import os
import datetime as dt
DB_USER = "root"
DB_PASSWORD = "micolash12"
DB_HOST = "localhost"
DATABASE = "globant_db"
DB_PORT = 3306
connect_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DATABASE}?charset=utf8"
engine = create_engine(connect_string)
# Create the connection up and running
def get_all_files(dir: str) -> list:
    # Return all files names from the data dir without extension
    files = [f.replace(".csv", "") for f in os.listdir(dir) if not f.startswith('.')]
    return files

async def upload_csv(file: UploadFile = File(...))->str:
    # create a path to store the file in the data directory
    file_path = os.path.join("data", file.filename)

    # open the file and write the contents to the specified path
    with open(file_path, "wb") as buffer:
        buffer.write(await file.read())

    return {"filename": file.filename}
# Create the service of reading the csv
## ToDo: Remove
def read_csv(file: UploadFile = File()) -> pd.DataFrame:
    df = pd.read_csv(file)
    return df


# Upload the file to the database (batch)
def upload_df_to_sql(df: pd.DataFrame, table_name: str) -> None:

    # Use the built-in function 'to_sql' to write the dataframe to the database
    df.to_sql(table_name, engine, if_exists="replace",
              index=False, chunksize=1000)
    print(f"{table_name} csv upload to database")
    return None
def add_constraints(table_name:str)->None:
    # Use the engine to write the primary key constraint to the database
    with engine.connect() as con:
        # and in the case of 'hired_employees' we add also a foreign key constraint
        con.execute(sa.text(f"alter table {table_name} add primary key (id);"))
        if ("hired_employees_t" in table_name):
            con.execute(sa.text(
                f"ALTER TABLE {table_name} ADD CONSTRAINT department_fk FOREIGN KEY(department_id) references departments_t(id)"
            ))
            con.execute(sa.text(
                f"ALTER TABLE {table_name} ADD CONSTRAINT job_fk FOREIGN KEY(job_id) references jobs_t(id)"
            ))

schema_hired = {
    "id":int,
    "name":str,
    "datetime":dt.datetime,
    "department_id":int,
    "job_id":int
}
schema_jobs = {
    "job_id":int,
    "job":str
}
schema_department = {
    "department_id":int,
    "department":str
}
schemas = [schema_hired,schema_jobs,schema_department]
def batch_upload():
    
    #Read all the csv files in the 'data' directory
    file_paths = [f"./data/{filename}.csv" for filename in get_all_files("data")]
    #Re arrange the file paths in order to execute first with 'jobs' and 'departments' and after the
    # 'hired_employees' since the last one depends on the others.
    file_paths[1], file_paths[-1] = file_paths[-1], file_paths[1]
    [upload_df_to_sql(pd.read_csv(file_path,names=schema.keys()),Path(file_path).stem ) for file_path,schema in zip(file_paths,schemas)]
    


In [6]:
batch_upload()


hired_employees csv upload to database
jobs csv upload to database
departments csv upload to database


In [7]:
file_paths = [f"./data/{filename}.csv" for filename in get_all_files("data")]
file_paths[1], file_paths[-1] = file_paths[-1], file_paths[1]
for file_path,schema in zip(file_paths,schemas):
    print(file_path,schema)

./data/hired_employees.csv {'id': <class 'int'>, 'name': <class 'str'>, 'datetime': <class 'datetime.datetime'>, 'department_id': <class 'int'>, 'job_id': <class 'int'>}
./data/jobs.csv {'job_id': <class 'int'>, 'job': <class 'str'>}
./data/departments.csv {'department_id': <class 'int'>, 'department': <class 'str'>}


In [13]:
schema_hired = {
    "id":"int64",
    "name":"object",
    "datetime":"object",
    "department_id":"int64",
    "job_id":"int64"
}
df = pd.read_csv("./data/hired_employees.csv",names=schema_hired.keys(),dtype=schema_hired,parse_dates=["datetime"])


print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   id             1999 non-null   Int64              
 1   name           1980 non-null   object             
 2   datetime       1985 non-null   datetime64[ns, UTC]
 3   department_id  1978 non-null   Int64              
 4   job_id         1983 non-null   Int64              
dtypes: Int64(3), datetime64[ns, UTC](1), object(1)
memory usage: 84.1+ KB
None


,id,name,datetime,department_id,job_id
0,1,Harold Vogt,2021-11-07 02:48:42+00:00,2,96
1,2,Ty Hofer,2021-05-30 05:43:46+00:00,8,<NA>
2,3,Lyman Hadye,2021-09-01 23:27:38+00:00,5,52
3,4,Lotti Crowthe,2021-10-01 13:04:21+00:00,12,71
4,5,Gretna Lording,2021-10-10 22:22:17+00:00,6,80


In [16]:

df = pd.read_csv("./data/jobs.csv",names=schema_jobs.keys())
print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   job_id  183 non-null    int64 
 1   job     183 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.0+ KB
None


,job_id,job
0,1,Marketing Assistant
1,2,VP Sales
2,3,Biostatistician IV
3,4,Account Representative II
4,5,VP Marketing


In [14]:
df = pd.read_csv("./data/departments.csv",names=schema_department.keys())
print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  12 non-null     int64 
 1   department     12 non-null     object
dtypes: int64(1), object(1)
memory usage: 324.0+ bytes
None


,department_id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering
